<a href="https://colab.research.google.com/github/helmiputeranurnashriq/zoomcamp-personal-repo/blob/main/Copy_of_Homework_data_talks_club_data_extraction_and_ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [ ]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 14
generator = square_root_generator(limit)

output = []

for sqrt_value in generator:
    output.append(sqrt_value)
    print(sqrt_value)

print("Sum of the outputs = ",sum(output))


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989
3.7416573867739413
Sum of the outputs =  36.59621325393519


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [30]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

peoples = []
for person in people_1():
    print(person)
    peoples.append(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

for person in people_2():
    print(person)
    peoples.append(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [31]:
print(sum(person['Age'] for person in peoples))

353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [46]:
import dlt

pipeline = dlt.pipeline(
    pipeline_name= "ws_learning",
    destination= "duckdb",
    dataset_name="generators"
)

info = pipeline.run(people_1(), table_name = "people_1", write_disposition="replace")

print(info)

info = pipeline.run(people_2(), table_name = "people_2", write_disposition="merge", primary_key="id")

print(info)


Pipeline ws_learning load step completed in 0.18 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/ws_learning.duckdb location to store data
Load package 1708266242.535624 is LOADED and contains no failed jobs
Pipeline ws_learning load step completed in 0.45 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/ws_learning.duckdb location to store data
Load package 1708266243.1303425 is LOADED and contains no failed jobs


In [47]:
import duckdb
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

In [48]:
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_1            │
│ people_2            │
└─────────────────────┘

In [49]:
people_1 = conn.sql("SELECT * FROM people_1").df()
display(people_1)

,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1708266242.535624,L/p83CjUlbZbaw
1,2,Person_2,27,City_A,1708266242.535624,OjRZscyDvdEf6Q
2,3,Person_3,28,City_A,1708266242.535624,K9gIVr1QCfB4kQ
3,4,Person_4,29,City_A,1708266242.535624,lLS1/mL514N07g
4,5,Person_5,30,City_A,1708266242.535624,RVtnDv24NMal/A


In [50]:
people_2 = conn.sql("SELECT * FROM people_2").df()
display(people_2)

,id,name,age,city,occupation,_dlt_load_id,_dlt_id
0,8,Person_8,38,City_B,Job_8,1708266243.1303425,kR2sFeAtlmoR5w
1,5,Person_5,35,City_B,Job_5,1708266243.1303425,kPhU7mEjN+TlCQ
2,7,Person_7,37,City_B,Job_7,1708266243.1303425,S4cGRFkzqNIjpA
3,4,Person_4,34,City_B,Job_4,1708266243.1303425,KZmG0yNIR4GlQQ
4,3,Person_3,33,City_B,Job_3,1708266243.1303425,x2ftNOBMcr1b2g
5,6,Person_6,36,City_B,Job_6,1708266243.1303425,0Y/PvNK/1bANuA


In [51]:
age_sum = conn.sql("SELECT SUM(age) FROM people_2").df()
display(age_sum)

,sum(age)
0,213.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX